# SETUP THE ENVIORNMENT

In [ ]:
!pip install tensorflow[and-cuda]

In [ ]:
!pip install keras-nlp

In [ ]:
!pip install gradio

In [ ]:
!pip install psutil


# IMPORT PACKAGES

Import Keras and KerasNLP.

In [ ]:
import keras
import keras_nlp

# GET KAGGLE CREDENTIALS


In [4]:
# Installed Kaggle Hub to login

import kagglehub
kagglehub.login()

# LOAD TRAINING DATASET

In [19]:
import json

# Load data
data = []
file_path = 'final_laws.jsonl'

with open(file_path, 'r') as file:
    for line in file:
        features = json.loads(line)
        
        # Format the example with Question and Answer
        template = "Question:\n{Question}\n\nAnswer:\n{Answer}"
        
        # Add the formatted string to data
        data.append(template.format(**features))

# use 996 training examples, to keep it fast in the beginning used 100 examples
data = data[:996]

In [ ]:
#show the data head

# Print the first few examples (head) with entry numbers
for idx, item in enumerate(data[:5], start=1):  # Adjust the number as needed
    print(f"Entry {idx}:{item}")
    

## Inference before fine tuning

In this section, you will query the model with various prompts to see how it responds.


### Buy a Home Prompt

### Query the model for suggestions on how to buy a home.

# Load Model



In [ ]:

import keras_nlp

# Initialize the gemma_lm model
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_instruct_2b_en")


# Define the template for the prompt
template = "Question:\n{Question}\n\nAnswer:\n{Answer}"

# Format the prompt with a question and leave the answer blank for model generation
prompt = template.format(
    Question="How can I buy a new house?",
    Answer="",  # Leaving the answer blank for the model to generate it
)

# Generate the response using gemma_lm model
print(gemma_lm.generate(prompt, max_length=256))


The model just responds with a basic advice on how to buy a home. 

### ELI5 How to buy a Home Prompt

Prompt the model to explain buying a home  in terms simple enough that a child can understand.


In [ ]:
prompt = template.format(
    Question="Explain the process of buying a home in a way that a 5 year old child could understand.",
    Answer="",
)
print(gemma_lm.generate(prompt, max_length=256))

The responses contains words that might not be easy to understand for a child such as budget, contract, reject.

# FINE-TUNING CODE

In [ ]:
## LoRA Fine-tuning

#To get better responses from the model, fine-tune the model with Low Rank Adaptation (LoRA) using the real estate law for ontario canada dataset

#The LoRA rank determines the dimensionality of the trainable matrices that are added to the original weights of the LLM. It controls the expressiveness and precision of the fine-tuning adjustments.

#A higher rank means more detailed changes are possible, but also means more trainable parameters. A lower rank means less computational overhead, but potentially less precise adaptation.

#This tutorial uses a LoRA rank of 4. In practice, begin with a relatively small rank (such as 4, 8, 16). This is computationally efficient for experimentation. Train your model with this rank and evaluate the performance improvement on your task. Gradually increase the rank in subsequent trials and see if that further boosts performance.

# Enable LoRA for the model and set the LoRA rank to 4.
gemma_lm.backbone.enable_lora(rank=4)
gemma_lm.summary()

Note that enabling LoRA reduces the number of trainable parameters significantly (from 2.5 billion to 1.3 million).

In [ ]:
# we will continue training from this point
import json
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Load data
data = []
file_path = './kaggle/input/mydata-final-laws/mydata/final_laws.jsonl'

with open(file_path, 'r') as file:
    for line in file:
        features = json.loads(line)
        
        # Format the example with Question and Answer
        template = "Question:\n{Question}\n\nAnswer:\n{Answer}"
        
        # Add the formatted string to data
        data.append(template.format(**features))

# Only use 100 training examples, to keep it fast
data = data[:996]

# Limit the input sequence length to 512 (to control memory usage).
gemma_lm.preprocessor.sequence_length = 512

# Use AdamW (a common optimizer for transformer models) with a slightly higher learning rate
optimizer = tf.keras.optimizers.AdamW(
    learning_rate=5e-5,  # Increased learning rate for faster convergence
    weight_decay=0.01,
)

# Exclude layernorm and bias terms from decay.
optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])

# Compile the model
gemma_lm.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=optimizer,
    weighted_metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

# Define EarlyStopping callback to monitor 'sparse_categorical_accuracy'
early_stopping = EarlyStopping(
    monitor='sparse_categorical_accuracy',  # Monitor accuracy
    patience=3,  # Stop after 3 epochs of no improvement
    mode='max',  # Stop when accuracy stops increasing
    restore_best_weights=True  # Restore weights from the epoch with the best performance
)

# Learning rate scheduler to reduce learning rate if accuracy stops improving
lr_scheduler = ReduceLROnPlateau(
    monitor='sparse_categorical_accuracy',
    factor=0.5,  # Reduce learning rate by half when accuracy plateaus
    patience=2,  # Number of epochs to wait before reducing learning rate
    min_lr=1e-7  # Set a minimum learning rate to prevent over-reduction
)

# Fit the model with early stopping and learning rate scheduler
gemma_lm.fit(
    data, 
    epochs=5, 
    batch_size=2,  # Increased batch size for better gradient updates
    callbacks=[early_stopping, lr_scheduler]  # Added learning rate scheduler
)


# NOT USED FINE-TUNING CODE

## Save the model after it has been fine-tuned for resume later

In [ ]:
# import tensorflow as tf
# from tensorflow import keras
# from tensorflow.keras import layers


# Save the Keras model in .keras format
gemma_lm.save('./kaggle/input/my_model.keras')

## Load the model after it has been fine-tuned


In [ ]:
# import keras so it is defined for later use
import keras
import keras_nlp

# Load the saved model in .keras format# CODE 11-30-2024 EDITS
# this is loading the model from Kaggle Public Upload link
# Import necessary libraries
import keras
import urllib.request

# Public URL of your model's .keras file
model_url = "https://www.kaggleusercontent.com/joelbest/gemma2b_re/keras/my_model_keraspreset/my_model.keras"

# Download the model file to the local environment
local_model_path = "my_model.keras"
urllib.request.urlretrieve(model_url, local_model_path)
print(f"Model downloaded to: {local_model_path}")

# Load the saved model in .keras format
gemma_lm = keras.models.load_model(local_model_path)

# Confirm the model is loaded successfully
print(gemma_lm.summary())
gemma_lm = keras.models.load_model('my_model.keras')



In [ ]:
# CODE 11-30-2024 EDITS
# this is loading the model from Kaggle Public Upload link
# Import necessary libraries
import keras
import urllib.request

# Public URL of your model's .keras file
model_url = "https://www.kaggleusercontent.com/joelbest/gemma2b_re/keras/my_model_keraspreset/my_model.keras"

# Download the model file to the local environment
local_model_path = "my_model.keras"
urllib.request.urlretrieve(model_url, local_model_path)
print(f"Model downloaded to: {local_model_path}")

# Load the saved model in .keras format
gemma_lm = keras.models.load_model(local_model_path)

# Confirm the model is loaded successfully
print(gemma_lm.summary())


## Save the model after it has been fine-tuned for resume later

# DOWNLOAD MY PUBLIC KAGGLE MODEL

In [ ]:
# 11-30-2024 edits 
import kagglehub
kagglehub.login()

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.model_download("joelbest/gemma2b_re/keras/my_model_keraspreset/v1/my_model.keras")

print("Path to model files:", path)

# TESTING

In [ ]:
#testing
import keras
import keras_nlp
# Load the saved model with the built-in tokenizer and language model
gemma_lm = keras.models.load_model('my_model.keras')

# Example input texts for inference
texts = [
    
    "Do I need a lawyer to handle the closing process when purchasing a property in Ontario?",
    "What are the legal implications of a co-ownership agreement when buying a property with someone else?",
    "Can I negotiate a lower real estate commission rate with my agent?",
    "What should I know about the rights and responsibilities for maintaining a shared fence between properties?"
]

# Set a smaller batch size
batch_size = 2

# Perform inference in batches
for i in range(0, len(texts), batch_size):
    batch_texts = texts[i:i + batch_size]

       
  # Use the model to generate answers directly
    answers = gemma_lm.generate(batch_texts)
    
    # Print the input questions and the generated answers
    for text, answer in zip(batch_texts, answers):
        print(f"Question: {text}")
        print(f"Answer: {answer}\n")



In [ ]:
import tensorflow as tf# List available GPUs and their details
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        details = tf.config.experimental.get_memory_info(gpu.name)
        print(f"GPU: {gpu.name}")
        print(f"  Current Memory Usage: {details['current']} bytes")
        print(f"  Peak Memory Usage: {details['peak']} bytes")
else:
    print("No GPU devices found.")

In [ ]:
# SAVE THE MODEL AGAIN #

gemma_lm.save('./kaggle/input/my_model.keras')

In [ ]:
# check the history of epochs trained so far 
# Assuming you have the history object from model.fit()
history = model.fit(...)

# To get the number of epochs that were run
num_epochs = len(history.epoch)
print(f"Number of epochs run: {num_epochs}")

## INFERENCE AFTER FINE TUNING
After fine-tuning, responses follow the instruction provided in the prompt.

### Buy a home Prompt


In [ ]:
prompt = template.format(
    Question="How can I buy a new house?",
    Answer="",
)
print(gemma_lm.generate(prompt, max_length=256))

The model now explains how to buy a home in Ontario.

### ELI5 How to Buy a home Prompt


In [ ]:
prompt = template.format(
    Question="Explain the process of buying a home in a way that a child could understand.",
crew     Answer="",
)
print(gemma_lm.generate(prompt, max_length=256))

## REMOVE THIS Note that for demonstration purposes, this tutorial fine-tunes the model on a small subset of the dataset for just one epoch and with a low LoRA rank value. To get better responses from the fine-tuned model, you can experiment with:

1. Increasing the size of the fine-tuning dataset
2. Training for more steps (epochs)
3. Setting a higher LoRA rank
4. Modifying the hyperparameter values such as `learning_rate` and `weight_decay`.


In [ ]:
#this code works

import gradio as gr
import tensorflow as tf
import keras

def generate_response(model: keras.Model, question: str, max_length: int = 250) -> str:
    """
    Generates a response for a single question using the provided model.

    Args:
        model (keras.Model): The fine-tuned language model.
        question (str): The user's input question.
        max_length (int): Maximum length of the generated answer.

    Returns:
        str: The generated answer text.
    """
    prompt_template = "Question:\n{question}\n\nAnswer:\n"
    prompt = prompt_template.format(question=question)

    try:
        # Generate response using the model's generate method
        generated_response = model.generate(prompt, max_length=max_length)

        # Decode the generated response based on its type
        if isinstance(generated_response, tf.Tensor):
            generated_text = generated_response.numpy().decode('utf-8')
        elif isinstance(generated_response, bytes):
            generated_text = generated_response.decode('utf-8')
        else:
            generated_text = generated_response  # Assume it's already a string

        # Extract the answer part by removing the prompt
        answer_text = generated_text[len(prompt):].strip()
        return answer_text

    except Exception as e:
        return f"⚠️ Error generating answer: {e}"

# Define the chatbot function for Gradio
def chatbot_gradio(user_input, model):
    """
    Gradio-compatible chatbot function.
    
    Args:
        user_input (str): The user's input question.
        model (keras.Model): The fine-tuned language model.

    Returns:
        str: The chatbot's answer.
    """
    if not user_input.strip():
        return "Chatbot: I'm here to help! Please enter a question."

    answer = generate_response(model, user_input, max_length=250)
    return answer

# Example usage with Gradio:
if gemma_lm:
    # Create Gradio interface
    interface = gr.Interface(
        fn=lambda user_input: chatbot_gradio(user_input, gemma_lm),
        inputs="text",
        outputs="text",
        title="Gemma 2b Language Model Chatbot",
        description="Enter a question, and the chatbot will respond."
    )

    # Launch the interface
    interface.launch(share=True)
else:
    print("⚠️ Model not loaded. Skipping chatbot.")



# PUSH MODEL TO KAGGLE


In [ ]:
# PUSH THE MODEL TO KAGGLE


import keras
import keras_nlp

# Load the saved model in .keras format
gemma_lm = keras.models.load_model('my_model.keras')

# Fine tune the model here. Example: Gemma fine-tuned to
# speak like a pirate. See bit.ly/gemma-pirate-demo
# ...

In [ ]:
# Save the finetuned model as a KerasNLP preset.
gemma_lm.save_to_preset("my_model_KerasPreset")

# Upload the preset as a new model variant on Kaggle
# kaggle_uri = "kaggle://my_kaggle_usernam/gemma-pirate/keras/gemma-pirate-instruct-7b"
kaggle_uri = "kaggle://joelbest/gemma2b_RE/keras/my_model_KerasPreset"
keras_nlp.upload_preset(kaggle_uri, "./my_model_KerasPreset")


# LOAD MODEL FROM KAGGLE


In [6]:
# 11-30-2024 edits 
# Download latest version
path = kagglehub.model_download("joelbest/gemma2b_re/keras/my_model_keraspreset")

print("Path to model files:", path)


100%|██████████| 143/143 [00:00<00:00, 432kB/s]

100%|██████████| 1.38k/1.38k [00:00<00:00, 593kB/s]


  0%|          | 0.00/785 [00:00<?, ?B/s]


100%|██████████| 785/785 [00:00<00:00, 276kB/s][A



  0%|          | 0.00/9.34G [00:00<?, ?B/s]



100%|██████████| 591/591 [00:00<00:00, 615kB/s]




100%|██████████| 2.91k/2.91k [00:00<00:00, 4.04MB/s]



 25%|██▍       | 1.00M/4.04M [00:00<00:00, 5.65MB/s]
100%|██████████| 4.04M/4.04M [00:00<00:00, 16.8MB/s]

  0%|          | 8.00M/9.34G [00:00<04:54, 34.1MB/s]
  0%|          | 18.0M/9.34G [00:00<03:32, 47.2MB/s]
  0%|          | 27.0M/9.34G [00:00<03:01, 55.1MB/s]
  0%|          | 34.0M/9.34G [00:00<03:04, 54.1MB/s]
  0%|          | 41.0M/9.34G [00:00<03:04, 54.2MB/s]
  1%|          | 49.0M/9.34G [00:00<02:42, 61.4MB/s]
  1%|          | 57.0M/9.34G [00:01<02:39, 62.5MB/s]
  1%|          | 65.0M/9.34G [00:01<02:43, 60.8MB/s]
  1%|          | 73.0M/9.34G [00:01<03:08, 52.7MB/s]
  1%|          | 81.0M/9.34G [00:01<02:55, 56.6MB/s]
  1%|          | 89.0M/9.34G [00:01<03:22, 49.1MB/s]
  1%|          | 100M/9.34G [00:01<02:44, 60.4MB/s] 
  1%|          | 107M/9.34G [00:02<02:45, 59.8MB/s]
  1%|          | 118M/9.34G [00:02<02:17, 72.2MB/s]
  1%|▏         | 126M/9.34G [00:02<02:23, 68.9MB/s]
  1%|▏         | 134M/9.34G [00:02<02:34, 

Path to model files: /home/jupyter/.cache/kagglehub/models/joelbest/gemma2b_re/keras/my_model_keraspreset/1
